In [1]:
import numpy as np
from sklearn import datasets
import pandas as pd

In [2]:
data = pd.read_csv('iris_data.csv')
data['Species'] = data['Species'].astype('category').cat.codes
data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
data_x = data[[item for item in data.columns[:4]]]
data_y = data[['Species']]

In [4]:
def polybasis(data, degree=1, monomial = False):
    """
    type(x): pandas dataframe
    type(degree): int
    type(monomial): Bool
    
    returns: polybasis of x as pandas dataframe
    """
    col_names = data.columns
    x = np.asarray(data)
    row,col = data.shape
    
    if degree >= 3:
        print("This is not a smart polynomial routine.\
        You may get numerical problems with a degree of 3 or more")
    
    if col==1:
        monomial = True
    if degree > 1:
        if monomial:
            ini_data = data
            tmp_data = ini_data
            cc = ['1']
            for _ in range(1,degree):
                tmp_data*=ini_data
                x = np.concatenate((x,tmp_data),axis=1)
                cc.append(str(_+1))
        else:
            matarray = np.repeat(x[:,:,np.newaxis], degree, axis=2)
            for _ in range(1,degree):
                matarray[:, :, _] =np.power(x,_+1) 
            matarray = np.transpose(matarray,(0,2,1))
            
            x = matarray[:, :, col-1 ]
            
            ad0 = list(range(1,degree+1))
            ad = ad0
            ncol_mat0 = degree
            ncol_x = degree
            d0 = [str(item) for item in ad0]
            cc = d0

            for _ in range(col-2,-1,-1):
                index0 = list(range(1,ncol_mat0+1))*ncol_x
                index = [item for item in range(1,ncol_x+1) for _ in range(ncol_mat0)]
                newad = [ad0[index0[i]-1]+ad[index[i]-1] for i in range(len(index0))]
                retain = [ True if newad[i]<=degree else False for i in range(len(newad))]
                
                mat0 = matarray[:, :, _]
                tmp_index0 = [index0[i]-1 for i in range(len(index0)) if retain[i]]
                tmp_index = [index[i]-1 for i in range(len(index)) if retain[i]]
                if any(retain):
                    newmat = np.multiply(mat0[:, tmp_index0 ], x[:,tmp_index])
                

                ddn = [ele+val for ele,val in zip([d0[ele] for ele in tmp_index0],\
                                                 [cc[ele] for ele in tmp_index])]        
                zeros = '0'*sum(retain)
                cc = ['0'+ele for ele in cc]
                d00 = [ele+zeros for ele in d0]
                x = np.concatenate((mat0,x,newmat), axis=1)
                cc = d00+cc+ddn
                ad = ad0+ad+ [newad[i] for i in range(len(newad)) if retain[i]]
                ncol_x = len(ad)
#             if dn:
        col_names = cc
    
    data = pd.DataFrame(x,columns=col_names)
    data['intercept'] = 1

    
    return data
        
        
        

In [5]:
# ret = polybasis(pd.DataFrame(data_x['Sepal.Length']),degree = 2)
# ret =  polybasis(data_x,degree =2)

In [6]:
# ret.head()

In [7]:
import numpy as np
from numpy.linalg import qr, inv, matrix_rank

from numpy.linalg import qr,lstsq, inv, matrix_rank



def polyreg(data_x, data_y, weights = [], degree = 1, monomial = False):
    """
    type(data_x): pandas dataframe
    type(data_y): pandas dataframe
    type(weights): list of weights
    type(degree): int
    type(monomial): bool
    
    returns dict of fitted vals, coefs and few more para
    """
    data_x = polybasis(data_x, degree, monomial)
    row,col = data_x.shape
    x_col = data_x.columns
    x = np.asarray(data_x)
    
#     y_col = data_y.columns
    y = np.asarray(data_y)
    
    if len(weights) == row:
        if any([True for ele in weights if ele<=0]):
            print("Only positive weights")
        
        weights = [ele**0.5 for ele in weights]
        y = np.multiply(y,np.reshape(np.array(weights),(row,1)))
        x = np.multiply(x,np.reshape(np.array(weights),(row,1)))
    
    q, r = qr(x)
    rank_r = matrix_rank(r)
    if rank_r < col:
        print("Didn't handle this case yet")
        ##inverse would not be possible
        ##gives error: LinAlgError: Incompatible dimensions :: but not in R
        
    coef = np.dot(np.dot(inv(r), q.T), y) # betas: R^-1 Q^T y
    fitted = np.dot(x, coef)
    if len(weights) == row:
        fitted = fitted/w
        
    return {'fitted': fitted,
           'coefficients': coef,
           'degree': degree,
           'monomial': monomial,
           'df': rank_r}
        
     
    
        
    

In [8]:
# ret = polybasis(data_x)
# polyreg(data_x,data_y)

In [9]:
# ret.head()


## contr.fda

In [10]:
from patsy import Helmert
from numpy.linalg import qr

In [11]:
t=Helmert()
theta = t.code_without_intercept(levels = np.unique(data_y)).matrix

dp = np.array([1/3]*3)



In [12]:
def contr_fda(p,contrasts_):
    """
    type(p): numpy array
    type(contrasts_): numpy array of helmert contrasts
    returns np array
    """
    dim = len(p)
    sqp = np.sqrt(p/np.sum(p))
    x = np.append(np.array([[1]*dim]).T,contrasts_,1)*sqp
    tmp_y = np.append(np.zeros((1,dim-1)),np.identity(dim-1),0)
    tmp_deno = np.array([sqp]*(dim-1)).T
    return np.dot(qr(x)[0],tmp_y)/tmp_deno
    
    

## fda

In [66]:
from numpy.linalg import svd

def fda(df_x,df_y):
    """
    type(df_x): pandas dataframe
    type(df_y): pandas dataframe
    """
    x_dim,y_dim = df_x.shape
    t=Helmert()
    theta = t.code_without_intercept(levels = np.unique(df_y)).matrix
    prior = np.array([1/3]*3)
    
    theta = contr_fda(prior,theta)
    
    level_counts = df_y[df_y.columns[0]].value_counts().sort_index()
    lvl_count = 0
    i=0
#     print(df_y.shape[0],level_counts.shape[0]-1)
    Theta = np.empty((df_y.shape[0],level_counts.shape[0]-1))
    for item in level_counts:
        Theta[lvl_count:lvl_count+item] = theta[level_counts.index[i]]
        i+=1
        lvl_count+=item
    
    fit =  polyreg(df_x,Theta)
    ssm = np.matmul(Theta.T, (fit['fitted']/x_dim))
    thetan, lambd, _ = svd(ssm)
    
    discr_eigen = lambd/(1-lambd)
    pe = 100*np.cumsum(discr_eigen)/np.sum(discr_eigen)
    
    ##not yet implemented
#     dimension = lambd.size
    dimension = 3
    alpha = np.sqrt(lambd)
    sgima = np.sqrt(1-lambd)
    means = np.matmul(theta, thetan)*sgima/alpha
    return {
        "percent_explained": pe,
        'values' : lambd,
        'means' : means,
        'theta_mod' : thetan,
        'dimention' : dimension,
        'prior' : prior,
        'fit': fit
    }
    
    
    
    

In [67]:
fda(data_x,data_y)

{'dimention': 3, 'fit': {'coefficients': array([[ 0.10555274, -0.09731787],
         [ 0.84319291,  0.4301367 ],
         [-0.54541113,  0.00845964],
         [ 0.53501004,  1.17050071],
         [-1.78670201, -2.18202234]]),
  'degree': 1,
  'df': 5,
  'fitted': array([[ 1.04621860e+00, -9.26921399e-01],
         [ 6.03511595e-01, -1.12252618e+00],
         [ 8.05580741e-01, -1.01788122e+00],
         [ 6.01623950e-01, -1.04947118e+00],
         [ 1.11998262e+00, -8.74175942e-01],
         [ 1.35854026e+00, -5.47424048e-01],
         [ 9.62623940e-01, -8.04226063e-01],
         [ 8.96802921e-01, -9.59357319e-01],
         [ 4.66415932e-01, -1.11688091e+00],
         [ 5.79788769e-01, -1.19571661e+00],
         [ 1.19198189e+00, -8.69243457e-01],
         [ 8.21151259e-01, -9.39047781e-01],
         [ 5.39455316e-01, -1.22984446e+00],
         [ 6.50302283e-01, -1.18372341e+00],
         [ 1.65078420e+00, -7.81667486e-01],
         [ 1.92088476e+00, -3.63242986e-01],
         [ 1.57670

In [22]:
# np.matmul(Theta.T,method_output['fitted']/150)
from numpy.linalg import svd

In [37]:
np.matmul(np.identity(2),np.array(([4,2],[2,10]))/2)

array([[2., 1.],
       [1., 5.]])

In [60]:
100*np.cumsum(method_output[1]/(1-method_output[1]))/np.sum(method_output[1]/(1-method_output[1]))

array([ 99.1212605, 100.       ])